<a href="https://colab.research.google.com/github/anne030303/data-course-sample/blob/main/sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [39]:
import pandas as pd
import gzip, json
import datetime

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [40]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-26 05:24:01--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.1’

All_Beauty.csv.1    100%[===================>]  14.78M  23.5MB/s    in 0.6s    

2021-12-26 05:24:02 (23.5 MB/s) - ‘All_Beauty.csv.1’ saved [15499476/15499476]

--2021-12-26 05:24:02--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.1’

meta_All_Beauty.jso 100%[===================>]   9.85M  18.6MB/s    in 0.5s   

In [41]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [42]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [43]:
# metadata_new = metadata[['asin', 'brand', 'title', 'price', 'rank', 'description', 'also_view', 'also_buy']].copy()
# metadata_new.loc[:,'rank_num'] = metadata_new['rank'].str.split('in', n = 1, expand = True)[0].str.replace(',','')
# metadata_new.loc[:,'rank_cat'] = metadata_new['rank'].str.split('in', n = 1, expand = True)[1].str.replace('(','')
# metadata_new.loc[:,'price'] = pd.to_numeric(metadata_new['price'].str.replace('$',''), errors='coerce')
# metadata_new.head()

In [44]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [47]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

In [48]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]

ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [49]:
# ratings_by_asin = ratings_trainings.groupby('asin').mean()['overall'].reset_index()[['asin', 'overall']]#.set_index('asin')
# ratings_sku_detail = pd.merge(ratings_by_asin,metadata_new)
# ratings_sku_detail.head()
ratings_trainings.head()

,asin,reviewerID,overall,unixReviewTime,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,1407628800,2014-08-10
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000,2013-03-11
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200,2011-12-25


In [50]:
#第一組：只抓近60天的資料、第二組：只抓去年同月的資料、第三組：近60天的資料與去年同月資料各半
def process(ratings_trainings,k):
  ratings_k_list = []

  ratings_1 = ratings_trainings[(ratings_trainings['DATE'] + datetime.timedelta(days = 60)) > '2018-09-01']
  ratings_k1 = list(ratings_1['asin'].value_counts()[:k].to_dict().keys())
  ratings_k_list.append(ratings_k1)

  ratings_2 = ratings_trainings[(ratings['DATE'] >= '2017-09-01') & (ratings['DATE'] <= '2017-09-30')]
  ratings_k2 = list(ratings_2['asin'].value_counts()[:k].to_dict().keys())
  ratings_k_list.append(ratings_k2)

  k1 = int(k/2)
  k2 = k-k1
  ratings_k_list.append(ratings_k1[:k1] + ratings_k2[:k2])
  
  return ratings_k_list
ratings_k_list = process(ratings_trainings,10)
ratings_k_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


[['B01DKQAXC0',
  'B00W259T7G',
  'B013XKHA4M',
  'B012Z7IHHI',
  'B01AVJCDYA',
  'B0195R1FT8',
  'B01CJNZKZK',
  'B00JVU3K9I',
  'B01DLR9IDI',
  'B018WCT01C'],
 ['B0010ZBORW',
  'B00W259T7G',
  'B01DKQAXC0',
  'B000050FDY',
  'B001LNODUS',
  'B019FWRG3C',
  'B001QY8QXM',
  'B002GP80EU',
  'B0195R1FT8',
  'B01BZV6VVY'],
 ['B01DKQAXC0',
  'B00W259T7G',
  'B013XKHA4M',
  'B012Z7IHHI',
  'B01AVJCDYA',
  'B0010ZBORW',
  'B00W259T7G',
  'B01DKQAXC0',
  'B000050FDY',
  'B001LNODUS']]

## 產生推薦

In [51]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    Your Code
    '''
    ratings_trainings = training_data
    recommendations = {user: ratings_trainings for user in users}
    
    #recommendations = {user: ratings_trainings['asin'].sample(n=k).tolist() for user in users}
    return recommendations
ratings_by_user_list = []
for ratings_k in ratings_k_list:
  ratings_by_user_list.append(recommender(ratings_k, users))
#ratings_by_user

## 結果評估

In [52]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score
for ratings_by_user in ratings_by_user_list:
  print(evaluate(ratings_testings_by_user, ratings_by_user))

0.15254237288135594
0.1
0.09661016949152543
